<a href="https://colab.research.google.com/github/rajitakolla/computational_methods/blob/master/In_class_exercise_07.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **The seventh in-class-exercise (20 points in total, 10/21/2020)**

Question description: In the last in-class-exercise (exercise-06), you collected the titles of 100 articles about data science, natural language processing, and machine learning. The 100 article titles will be used as the text corpus of this exercise. Perform the following tasks:

## (1) (8 points) Generate K topics by using LDA, the number of topics K should be decided by the coherence score, then summarize what are the topics. You may refer the code here: 

https://www.machinelearningplus.com/nlp/topic-modeling-gensim-python/

In [ ]:
# Write your code here
import nltk; 
nltk.download('stopwords')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


True

In [ ]:
!pip install spacy

In [ ]:
!pip install pyLDAvis

     |████████████████████████████████| 1.6MB 1.4MB/s 
  Created wheel for pyLDAvis: filename=pyLDAvis-2.1.2-py2.py3-none-any.whl size=97712 sha256=952a83af57c2ec4e8c816d67dc8fbec2eb2f361b0cac136febd39af20379899b
  Stored in directory: /root/.cache/pip/wheels/98/71/24/513a99e58bb6b8465bae4d2d5e9dba8f0bef8179e3051ac414
Successfully built pyLDAvis


In [ ]:
import re
import numpy as np
import pandas as pd
from pprint import pprint

# Gensim
import gensim
import gensim.corpora as corpora
from gensim.utils import simple_preprocess
from gensim.models import CoherenceModel

# spacy for lemmatization
import spacy

# Plotting tools
import pyLDAvis
import pyLDAvis.gensim  # don't skip this
import matplotlib.pyplot as plt
%matplotlib inline

# Enable logging for gensim - optional
import logging
logging.basicConfig(format='%(asctime)s : %(levelname)s : %(message)s', level=logging.ERROR)

import warnings
warnings.filterwarnings("ignore",category=DeprecationWarning)

In [ ]:
# NLTK Stop words
from nltk.corpus import stopwords
stop_words = stopwords.words('english')

In [ ]:
import pandas as pd

df = pd.read_csv("/content/q1_data.csv")
#display only 10
df.head(5)

,Title,Author
0,Data science in action,"W Van Der Aalst - Process mining, 2016 - Springer"
1,Data science and its relationship to big data ...,"F Provost, T Fawcett - Big data, 2013 - lieber..."
2,Data Science for Business: What you need to kn...,"F Provost, T Fawcett - 2013 - books.google.com"
3,Data science and prediction,"V Dhar - Communications of the ACM, 2013 - dl...."
4,"Data science, predictive analytics, and big da...","MA Waller, SE Fawcett - Journal of Business Lo..."


In [ ]:
df["Title"]=df["Title"].str.lower()
# Convert to list
data = df["Title"].values.tolist()

#remove end spacings
data = [re.sub(" $","",sent) for sent in data]

# Remove Emails
data = [re.sub('\S*@\S*\s?', '', sent) for sent in data]

# Remove new line characters
data = [re.sub('\s+', ' ', sent) for sent in data]

# Remove distracting single quotes
data = [re.sub("\'", "", sent) for sent in data]

print(data[:5])

['data science in action', 'data science and its relationship to big data and data-driven decision making', 'data science for business: what you need to know about data mining and data-analytic thinking', 'data science and prediction', 'data science, predictive analytics, and big data: a revolution that will transform supply chain design and management']


In [ ]:
def sent_to_words(sentences):
    for sentence in sentences:
        yield(gensim.utils.simple_preprocess(str(sentence), deacc=True))  # deacc=True removes punctuations

data_words = list(sent_to_words(data))

print(data_words[:5])

[['data', 'science', 'in', 'action'], ['data', 'science', 'and', 'its', 'relationship', 'to', 'big', 'data', 'and', 'data', 'driven', 'decision', 'making'], ['data', 'science', 'for', 'business', 'what', 'you', 'need', 'to', 'know', 'about', 'data', 'mining', 'and', 'data', 'analytic', 'thinking'], ['data', 'science', 'and', 'prediction'], ['data', 'science', 'predictive', 'analytics', 'and', 'big', 'data', 'revolution', 'that', 'will', 'transform', 'supply', 'chain', 'design', 'and', 'management']]


In [ ]:
# Build the bigram and trigram models
bigram = gensim.models.Phrases(data_words, min_count=5, threshold=100) # higher threshold fewer phrases.
trigram = gensim.models.Phrases(bigram[data_words], threshold=100)  

# Faster way to get a sentence clubbed as a trigram/bigram
bigram_mod = gensim.models.phrases.Phraser(bigram)
trigram_mod = gensim.models.phrases.Phraser(trigram)

# See trigram example
print(trigram_mod[bigram_mod[data_words[0]]])

['data', 'science', 'in', 'action']


In [ ]:
# Define functions for stopwords, bigrams, trigrams and lemmatization
def remove_stopwords(texts):
    return [[word for word in simple_preprocess(str(doc)) if word not in stop_words] for doc in texts]

def make_bigrams(texts):
    return [bigram_mod[doc] for doc in texts]

def make_trigrams(texts):
    return [trigram_mod[bigram_mod[doc]] for doc in texts]

def lemmatization(texts, allowed_postags=['NOUN', 'ADJ', 'VERB', 'ADV']):
    """https://spacy.io/api/annotation"""
    texts_out = []
    for sent in texts:
        doc = nlp(" ".join(sent)) 
        texts_out.append([token.lemma_ for token in doc if token.pos_ in allowed_postags])
    return texts_out

In [ ]:
# Remove Stop Words
data_words_nostops = remove_stopwords(data_words)

# Form Bigrams
data_words_bigrams = make_bigrams(data_words_nostops)

# Initialize spacy 'en' model, keeping only tagger component (for efficiency)
# python3 -m spacy download en
nlp = spacy.load('en', disable=['parser', 'ner'])

# Do lemmatization keeping only noun, adj, vb, adv
data_lemmatized = lemmatization(data_words_bigrams, allowed_postags=['NOUN', 'ADJ', 'VERB', 'ADV'])

print(data_lemmatized[:1])

[['datum', 'science', 'action']]


In [ ]:
# Create Dictionary
id2word = corpora.Dictionary(data_lemmatized)

# Create Corpus
texts = data_lemmatized

# Term Document Frequency
corpus = [id2word.doc2bow(text) for text in texts]

# View
print(corpus[:1])

[[(0, 1), (1, 1), (2, 1)]]


In [ ]:
res = [[(id2word[id], freq) for id, freq in cp] for cp in corpus[:]]

In [ ]:
lda_model = gensim.models.ldamodel.LdaModel(corpus=corpus,id2word=id2word,num_topics=10,random_state=100,update_every=1,chunksize=100,passes=10,alpha='auto',per_word_topics=True)

In [ ]:
# Print the Keyword in the 10 topics
pprint(lda_model.print_topics())
doc_lda = lda_model[corpus]

[(0,
  '0.049*"science" + 0.049*"datum" + 0.049*"data" + 0.025*"nuclear" + '
  '0.025*"big" + 0.025*"model" + 0.025*"cluster" + 0.013*"evaluate" + '
  '0.013*"next" + 0.013*"list"'),
 (1,
  '0.057*"new" + 0.030*"econometric" + 0.030*"trick" + 0.030*"big" + '
  '0.030*"analysis" + 0.030*"method" + 0.030*"management" + '
  '0.030*"infrastructure" + 0.003*"science" + 0.003*"datum"'),
 (2,
  '0.118*"datum" + 0.040*"science" + 0.030*"network" + 0.030*"protein" + '
  '0.030*"structure" + 0.021*"data" + 0.021*"algorithm" + 0.021*"model" + '
  '0.021*"stream" + 0.020*"citation"'),
 (3,
  '0.082*"science" + 0.051*"datum" + 0.026*"fair" + 0.026*"analyze" + '
  '0.026*"analysis" + 0.026*"product" + 0.026*"data" + 0.020*"computer" + '
  '0.014*"cloud" + 0.014*"revisit"'),
 (4,
  '0.092*"science" + 0.027*"workflow" + 0.027*"datum" + 0.015*"mining" + '
  '0.014*"method" + 0.014*"application" + 0.014*"reference" + 0.014*"industry" '
  '+ 0.014*"life" + 0.014*"experiment"'),
 (5,
  '0.096*"datum" + 0.

In [ ]:
# Compute Perplexity
print('\nPerplexity: ', lda_model.log_perplexity(corpus))  # a measure of how good the model is. lower the better.

# Compute Coherence Score
coherence_model_lda = CoherenceModel(model=lda_model, texts=data_lemmatized, dictionary=id2word, coherence='c_v')
coherence_lda = coherence_model_lda.get_coherence()
print('\nCoherence Score: ', coherence_lda)


Perplexity:  -6.181110519278697

Coherence Score:  0.4167618041817181


In [ ]:
!pip install --upgrade gensim

Requirement already up-to-date: gensim in /usr/local/lib/python3.6/dist-packages (3.8.3)


In [ ]:
import os       #importing os to set environment variable
def install_java():
  !apt-get install -y openjdk-8-jdk-headless -qq > /dev/null      #install openjdk
  os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-8-openjdk-amd64"     #set environment variable
  !java -version       #check java version
install_java()

openjdk version "11.0.8" 2020-07-14
OpenJDK Runtime Environment (build 11.0.8+10-post-Ubuntu-0ubuntu118.04.1)
OpenJDK 64-Bit Server VM (build 11.0.8+10-post-Ubuntu-0ubuntu118.04.1, mixed mode, sharing)


In [ ]:
!wget http://mallet.cs.umass.edu/dist/mallet-2.0.8.zip
!unzip mallet-2.0.8.zip

--2020-10-23 00:05:43--  http://mallet.cs.umass.edu/dist/mallet-2.0.8.zip
Resolving mallet.cs.umass.edu (mallet.cs.umass.edu)... 128.119.246.70
Connecting to mallet.cs.umass.edu (mallet.cs.umass.edu)|128.119.246.70|:80... connected.
HTTP request sent, awaiting response... 200 OK
Length: 16184794 (15M) [application/zip]
Saving to: ‘mallet-2.0.8.zip.2’

mallet-2.0.8.zip.2  100%[===================>]  15.43M  5.87MB/s    in 2.6s    

2020-10-23 00:05:47 (5.87 MB/s) - ‘mallet-2.0.8.zip.2’ saved [16184794/16184794]

Archive:  mallet-2.0.8.zip
replace mallet-2.0.8/bin/classifier2info? [y]es, [n]o, [A]ll, [N]one, [r]ename: all
error:  invalid response [all]
replace mallet-2.0.8/bin/classifier2info? [y]es, [n]o, [A]ll, [N]one, [r]ename: all
error:  invalid response [all]
replace mallet-2.0.8/bin/classifier2info? [y]es, [n]o, [A]ll, [N]one, [r]ename: all
error:  invalid response [all]
replace mallet-2.0.8/bin/classifier2info? [y]es, [n]o, [A]ll, [N]one, [r]ename: all
error:  invalid response [a

In [ ]:
os.environ['MALLET_HOME'] = '/content/mallet-2.0.8'
mallet_path = '/content/mallet-2.0.8/bin/mallet'
 
ldamallet = gensim.models.wrappers.LdaMallet(mallet_path, corpus=corpus, num_topics=10, id2word=id2word)

In [ ]:
# Show Topics
pprint(ldamallet.show_topics(formatted=False))

# Compute Coherence Score
coherence_model_ldamallet = CoherenceModel(model=ldamallet, texts=data_lemmatized, dictionary=id2word, coherence='c_v')
coherence_ldamallet = coherence_model_ldamallet.get_coherence()
print('\nCoherence Score: ', coherence_ldamallet)

[(5,
  [('data', 0.23529411764705882),
   ('method', 0.17647058823529413),
   ('system', 0.058823529411764705),
   ('text', 0.058823529411764705),
   ('promise', 0.058823529411764705),
   ('summarize', 0.058823529411764705),
   ('econometric', 0.058823529411764705),
   ('prediction', 0.058823529411764705),
   ('material', 0.058823529411764705),
   ('action', 0.058823529411764705)]),
 (17,
  [('protein', 0.3333333333333333),
   ('predict', 0.08333333333333333),
   ('statistical', 0.08333333333333333),
   ('mining', 0.08333333333333333),
   ('european', 0.08333333333333333),
   ('base', 0.08333333333333333),
   ('practice', 0.08333333333333333),
   ('cell', 0.08333333333333333),
   ('update', 0.08333333333333333),
   ('summarize', 0.0)]),
 (23,
  [('citizen', 0.1875),
   ('technology', 0.125),
   ('quality', 0.0625),
   ('integration', 0.0625),
   ('change', 0.0625),
   ('cloud', 0.0625),
   ('education', 0.0625),
   ('omega', 0.0625),
   ('plan', 0.0625),
   ('industry', 0.0625)]),
 (22

In [ ]:
def compute_coherence_values(dictionary, corpus, texts, start, limit, step):
    """
    Compute c_v coherence for various number of topics

    Parameters:
    ----------
    dictionary : Gensim dictionary
    corpus : Gensim corpus
    texts : List of input texts
    limit : Max num of topics

    Returns:
    -------
    model_list : List of LDA topic models
    coherence_values : Coherence values corresponding to the LDA model with respective number of topics
    """
    coherence_values = []
    model_list = []
    for num_topics in range(start, limit, step):
        model = gensim.models.wrappers.LdaMallet(mallet_path, corpus=corpus, num_topics=num_topics, id2word=id2word)
        model_list.append(model)
        coherencemodel = CoherenceModel(model=model, texts=texts, dictionary=dictionary, coherence='c_v')
        coherence_values.append(coherencemodel.get_coherence())

    return model_list, coherence_values

In [ ]:
# Can take a long time to run.
model_list, coherence_values = compute_coherence_values(dictionary=id2word, corpus=corpus, texts=data_lemmatized, start=2, limit=40, step=6)

In [ ]:
# Show graph
limit=40; start=2; step=6;
x = range(start, limit, step)

In [ ]:
for m, cv in zip(x, coherence_values):
    print("Num Topics =", m, " has Coherence Value of", round(cv, 4))

Num Topics = 2  has Coherence Value of 0.6618
Num Topics = 8  has Coherence Value of 0.7501
Num Topics = 14  has Coherence Value of 0.7702
Num Topics = 20  has Coherence Value of 0.7684
Num Topics = 26  has Coherence Value of 0.7417
Num Topics = 32  has Coherence Value of 0.7363
Num Topics = 38  has Coherence Value of 0.7251


In [ ]:
# Select the model and print the topics
optimal_model = model_list[5]
model_topics = optimal_model.show_topics(formatted=False)
pprint(optimal_model.print_topics(num_words=10))

[(23,
  '0.267*"structure" + 0.133*"discovery" + 0.067*"role" + 0.067*"orbiter" + '
  '0.067*"make" + 0.067*"intensive" + 0.067*"unrealized" + 0.067*"transform" + '
  '0.067*"source" + 0.067*"guiding"'),
 (22,
  '0.333*"fair" + 0.111*"principle" + 0.111*"scientific" + 0.111*"customer" + '
  '0.111*"decay" + 0.111*"social" + 0.111*"objective" + 0.000*"cipre" + '
  '0.000*"gateway" + 0.000*"holographic"'),
 (17,
  '0.556*"science" + 0.111*"implication" + 0.111*"detect" + '
  '0.111*"integrative" + 0.111*"facility" + 0.000*"trap" + 0.000*"holographic" '
  '+ 0.000*"technique" + 0.000*"summarize" + 0.000*"multivariate"'),
 (14,
  '0.417*"science" + 0.083*"enable" + 0.083*"hipe" + 0.083*"pipeline" + '
  '0.083*"vii" + 0.083*"material" + 0.083*"action" + 0.083*"prediction" + '
  '0.000*"holographic" + 0.000*"technique"'),
 (29,
  '0.267*"model" + 0.133*"data" + 0.067*"international" + 0.067*"challenge" + '
  '0.067*"promise" + 0.067*"predictive" + 0.067*"age" + 0.067*"ranking" + '
  '0.067*"

## (2) (8 points) Generate K topics by using LSA, the number of topics K should be decided by the coherence score, then summarize what are the topics. You may refer the code here:

https://www.datacamp.com/community/tutorials/discovering-hidden-topics-python

In [ ]:
number_of_topics = 10

In [ ]:
def prepare_corpus(doc_clean):
    """
    Input  : clean document
    Purpose: create term dictionary of our courpus and Converting list of documents (corpus) into Document Term Matrix
    Output : term dictionary and Document Term Matrix
    """
    # Creating the term dictionary of our courpus, where every unique term is assigned an index. dictionary = corpora.Dictionary(doc_clean)
    dictionary = corpora.Dictionary(doc_clean)
    # Converting list of documents (corpus) into Document Term Matrix using dictionary prepared above.
    doc_term_matrix = [dictionary.doc2bow(doc) for doc in doc_clean]
    # generate LDA model 
    return dictionary,doc_term_matrix

In [ ]:
from gensim.models import LsiModel
def create_gensim_lsa_model(doc_clean,number_of_topics,words):
    """
    Input  : clean document, number of topics and number of words associated with each topic
    Purpose: create LSA model using gensim
    Output : return LSA model
    """
    dictionary,doc_term_matrix=prepare_corpus(doc_clean)
    # generate LSA model
    lsamodel = LsiModel(doc_term_matrix, num_topics=number_of_topics, id2word = dictionary)  # train model
    print(lsamodel.print_topics(num_topics=number_of_topics, num_words=words))
    return lsamodel

In [ ]:
def compute_coherence_values(dictionary, doc_term_matrix, doc_clean, stop, start, step):
    """
    Input   : dictionary : Gensim dictionary
              corpus : Gensim corpus
              texts : List of input texts
              stop : Max num of topics
    purpose : Compute c_v coherence for various number of topics
    Output  : model_list : List of LSA topic models
              coherence_values : Coherence values corresponding to the LDA model with respective number of topics
    """
    coherence_values = []
    model_list = []
    for num_topics in range(start, stop, step):
        # generate LSA model
        model = LsiModel(doc_term_matrix, num_topics=number_of_topics, id2word = dictionary)  # train model
        model_list.append(model)
        coherencemodel = CoherenceModel(model=model, texts=doc_clean, dictionary=dictionary, coherence='c_v')
        coherence_values.append(coherencemodel.get_coherence())
    return model_list, coherence_values

In [ ]:
dictionary,doc_term_matrix=prepare_corpus(data_lemmatized)
model_list_lsa, coherence_values_lsa = compute_coherence_values(dictionary, doc_term_matrix,data_lemmatized,44,2,6)

In [ ]:
coherence_values_lsa

[0.4062175633589294,
 0.4399488129155105,
 0.4399488129155105,
 0.4062175633589294,
 0.4134232436240143,
 0.4320821458925854,
 0.4062175633589294]

In [ ]:
# Write your code here
for m, cv in zip(x, coherence_values_lsa):
    print("Num Topics =", m, " has Coherence Value of", round(cv, 4))

Num Topics = 2  has Coherence Value of 0.4062
Num Topics = 8  has Coherence Value of 0.4399
Num Topics = 14  has Coherence Value of 0.4399
Num Topics = 20  has Coherence Value of 0.4062
Num Topics = 26  has Coherence Value of 0.4134
Num Topics = 32  has Coherence Value of 0.4321
Num Topics = 38  has Coherence Value of 0.4062


In [ ]:
# Select the model and print the topics
optimal_model_lsa = model_list_lsa[5]
model_topics_lsa = optimal_model_lsa.show_topics(formatted=False)
pprint(optimal_model_lsa.print_topics(num_words=10))

[(0,
  '0.755*"datum" + 0.562*"science" + 0.106*"mining" + 0.096*"big" + '
  '0.082*"analytic" + 0.063*"knowledge" + 0.055*"relationship" + 0.054*"data" '
  '+ 0.053*"management" + 0.050*"business"'),
 (1,
  '0.744*"science" + -0.538*"datum" + -0.128*"big" + 0.085*"research" + '
  '0.079*"change" + -0.077*"protein" + -0.069*"analytic" + -0.069*"mining" + '
  '0.053*"scientific" + 0.052*"citizen"'),
 (2,
  '-0.622*"data" + -0.372*"nuclear" + -0.234*"vii" + -0.234*"technology" + '
  '-0.234*"endf" + -0.158*"structure" + -0.144*"product" + -0.139*"next" + '
  '-0.138*"generation" + -0.138*"evaluate"'),
 (3,
  '0.388*"data" + 0.378*"big" + 0.276*"analytic" + 0.245*"new" + '
  '-0.232*"nuclear" + -0.178*"technology" + -0.178*"endf" + -0.178*"vii" + '
  '-0.174*"protein" + 0.125*"structure"'),
 (4,
  '0.293*"protein" + 0.292*"data" + -0.274*"big" + 0.263*"network" + '
  '-0.245*"analytic" + -0.242*"new" + -0.220*"nuclear" + 0.186*"fair" + '
  '-0.161*"endf" + -0.161*"vii"'),
 (5,
  '0.341*"r

## (3) (4 points) Compare the results generated by the two topic modeling algorithms, which one is better? You should explain the reasons in details.

In [ ]:
# Write your answer here (no code needed for this question)